In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
#splitting the data
df=pd.read_csv('ML-Om1.csv')
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [13]:
Y_col = 'diagnosis'
X_cols = (df.loc[:, df.columns != Y_col].columns).drop('id')
print(type(X_cols))

<class 'pandas.core.indexes.base.Index'>


In [15]:
df1=df.dropna()
df1.shape

(563, 32)

In [16]:
def normal_dist(x, mean, stdev):
  return 1/(stdev * np.sqrt(2 * np.pi)) * np.exp( - (x - mean)**2 / (2 * stdev**2))

In [17]:
class Fishers_Linear_Discriminant_Analysis:
  def __init__(self, n_components=1):
    self.n_components = n_components #n_components = 1 for lda 
    self.w = None
    self.f_classes={}
    self.class_prob={}
    self.final_class_means={}
    self.final_class_stdev={}
    #self.thresh=0


  def fit(self, X, y):
    D_features = X.shape[1] # 500something,30
    class_labels = np.unique(y)

    #S_W(within class cov mat), S_B(between class cov matrix)
    S_W = np.zeros((D_features, D_features))   
    for c in class_labels:
      X_c = X[y == c]
      class_mean = np.mean(X_c, axis = 0)
      S_W += (X_c - class_mean).T.dot(X_c - class_mean) # (30,D)*(D,30)=(30,30) summing all the scatter matrices 

    X_negative = X[y==0]
    X_positive = X[y==1]
    negative_mean = np.mean(X_negative, axis=0)
    positive_mean = np.mean(X_positive, axis=0)
    negative_stdev = np.std(X_negative, axis = 0)
    positive_stdev = np.std(X_positive, axis = 0)
    self.w=np.linalg.inv(S_W) @ (negative_mean-positive_mean)

    X_projected = np.dot(X, self.w.T)
    for i in class_labels:
      self.f_classes[i]=X_projected[y==i]
    
    for i in self.f_classes:
      self.final_class_means[i]=np.mean(self.f_classes[i])
    
    for i in self.f_classes:
      self.final_class_stdev[i]=np.std(self.f_classes[i])
    
    for i in self.f_classes:
      self.class_prob[i]=len(self.f_classes[i])/(len(self.f_classes[0])+len(self.f_classes[1])) #p(c1) and p(c2)
    

    y1 = normal_dist(self.f_classes[0], self.final_class_means[0], self.final_class_stdev[0]) 
    y2 = normal_dist(self.f_classes[1], self.final_class_means[1], self.final_class_stdev[1])
    #plt.style.use('seaborn')
    #plt.figure(figsize = (6, 6))
    #plt.plot(self.f_classes[0], y1, color = 'blue')
    #plt.plot(self.f_classes[1], y2, color = 'red')
    #plt.scatter(X_negative, y1, marker = 'o', s = 25, color = 'magenta')
    #plt.scatter(X_positive, y2, marker = 'o', s = 25, color = 'cyan')
    plt.show()

  def predict(self, X):
    X_proj=np.dot (X, self.w.T)
    y_pred=[]
    for x in X_proj:
      prob_negative=normal_dist(x, self.final_class_means[0], self.final_class_stdev[0]) * self.class_prob[0]
      prob_positive=normal_dist(x, self.final_class_means[1], self.final_class_stdev[1]) * self.class_prob[1]
      prediction=(0 if prob_negative>prob_positive else 1)
      y_pred.append(prediction)
    
    return y_pred

  


**FLDM1**

In [18]:
def accuracy (y_pred, y_test):
  return np.sum (y_pred==y_test)/len(y_test)

In [24]:
def CM():
  TP=0
  FP=0
  TN=0
  FN=0
  yt=y_test.values.tolist()
  for i in range(len(y_pred)):
    if y_pred[i]==1 and yt[i]==1:
      TP+=1
    elif y_pred[i]==1 and yt[i]==0:
      FP+=1
    elif y_pred[i]==0 and yt[i]==1:
      FN+=1
    elif y_pred[i]==0 and yt[i]==0:
      TN+=1
  print("Confusion Matrix for FLDA1:")
  print(np.array([[TP,FP],[FN,TN]]))
  print(" ")
  print("Accuracy:", (TP + TN)/(TP + TN + FP + FN))
  print("Precision:", TP/(TP + FP))
  print("Recall:", TP/(TP + FN))

In [27]:
FLDA1 = Fishers_Linear_Discriminant_Analysis(1) # 1 is the no of dimensions u want to reduce ur d features to
for i in range (10):
  df_train = df1.sample(frac=0.67, random_state=i)
  df_test=df1.drop(df_train.index)
  X_train = df_train[X_cols]
  X_test = df_test[X_cols]
  y_train = (((df_train[Y_col]).apply(ord))/11-6).astype('int')
  y_test = (((df_test[Y_col]).apply(ord))/11-6).astype('int')
  FLDA1.fit(X_train, y_train)
  y_pred = np.array(FLDA1.predict(X_test))

  acc= accuracy(y_pred, y_test)
  print ("Accuracy for",i,"th train-test-split is:",acc*100,"%")
  # print(y_pred)
  # print(y_test)
  CM()
  print(" ")
  print(" ")

Accuracy for 0 th train-test-split is: 97.31182795698925 %
Confusion Matrix for FLDA1:
[[ 72   1]
 [  4 109]]
 
Accuracy: 0.9731182795698925
Precision: 0.9863013698630136
Recall: 0.9473684210526315
 
 
Accuracy for 1 th train-test-split is: 96.23655913978494 %
Confusion Matrix for FLDA1:
[[ 67   2]
 [  5 112]]
 
Accuracy: 0.9623655913978495
Precision: 0.9710144927536232
Recall: 0.9305555555555556
 
 
Accuracy for 2 th train-test-split is: 96.23655913978494 %
Confusion Matrix for FLDA1:
[[ 61   2]
 [  5 118]]
 
Accuracy: 0.9623655913978495
Precision: 0.9682539682539683
Recall: 0.9242424242424242
 
 
Accuracy for 3 th train-test-split is: 96.7741935483871 %
Confusion Matrix for FLDA1:
[[ 65   1]
 [  5 115]]
 
Accuracy: 0.967741935483871
Precision: 0.9848484848484849
Recall: 0.9285714285714286
 
 
Accuracy for 4 th train-test-split is: 95.6989247311828 %
Confusion Matrix for FLDA1:
[[ 72   0]
 [  8 106]]
 
Accuracy: 0.956989247311828
Precision: 1.0
Recall: 0.9
 
 
Accuracy for 5 th train-

**FLDA2**

In [28]:
df2=df1.sample(frac=1, axis=1).reset_index(drop=True)
X_cols2 = (df2.loc[:, df2.columns != Y_col].columns).drop('id')

In [30]:
FLDA2 = Fishers_Linear_Discriminant_Analysis(1) # 1 is the no of dimensions u want to reduce ur d features to
for i in range (10):
  df_train = df2.sample(frac=0.67, random_state=i)
  df_test=df2.drop(df_train.index)
  X_train2 = df_train[X_cols2]
  X_test2 = df_test[X_cols2]
  y_train = (((df_train[Y_col]).apply(ord))/11-6).astype('int')
  y_test = (((df_test[Y_col]).apply(ord))/11-6).astype('int')
  FLDA2.fit(X_train2, y_train)
  y_pred = np.array(FLDA2.predict(X_test2))

  acc= accuracy(y_pred, y_test)
  print ("Accuracy for",i,"th train-test-split is:",acc*100,"%")
  # print(y_pred)
  # print(y_test)
  CM()
  print(" ")
  print(" ")

Accuracy for 0 th train-test-split is: 97.31182795698925 %
Confusion Matrix for FLDA1:
[[ 72   1]
 [  4 109]]
 
Accuracy: 0.9731182795698925
Precision: 0.9863013698630136
Recall: 0.9473684210526315
 
 
Accuracy for 1 th train-test-split is: 96.23655913978494 %
Confusion Matrix for FLDA1:
[[ 67   2]
 [  5 112]]
 
Accuracy: 0.9623655913978495
Precision: 0.9710144927536232
Recall: 0.9305555555555556
 
 
Accuracy for 2 th train-test-split is: 96.23655913978494 %
Confusion Matrix for FLDA1:
[[ 61   2]
 [  5 118]]
 
Accuracy: 0.9623655913978495
Precision: 0.9682539682539683
Recall: 0.9242424242424242
 
 
Accuracy for 3 th train-test-split is: 96.7741935483871 %
Confusion Matrix for FLDA1:
[[ 65   1]
 [  5 115]]
 
Accuracy: 0.967741935483871
Precision: 0.9848484848484849
Recall: 0.9285714285714286
 
 
Accuracy for 4 th train-test-split is: 95.6989247311828 %
Confusion Matrix for FLDA1:
[[ 72   0]
 [  8 106]]
 
Accuracy: 0.956989247311828
Precision: 1.0
Recall: 0.9
 
 
Accuracy for 5 th train-